## 0. Instalacion de paquetes ###
La idea es setear el ambiente de codigo para poder empezar a trabajar con la consigna final de TP 

In [1]:
import pandas as pd # impotamos libreria pandas
import numpy as np #  importamos libreria numpy
import matplotlib as plt # importamos libreria matpoltlib
import seaborn as sns # importamos libreria seaborn
import scipy.stats as stats # importamos libreria scipy.stats

## 1. Carga del CSV con los datos electorales correspondientes ###
La idea es utilizar un csv de prueba para pdoer llevar adelante el analisis, el pedido del CSV se le hizo a ChatGPT para que pueda replicar el mismo

In [7]:
from pathlib import Path

# intentamos localizar el archivo en ../data con prefijo "encuesta"
data_dir = Path("../data")
if not data_dir.exists():
	raise FileNotFoundError(f"El directorio {data_dir.resolve()} no existe. Coloque el archivo dentro de ese directorio o ajuste la ruta.")

candidates = sorted(data_dir.glob("encuesta*"))
if not candidates:
	available = sorted([p.name for p in data_dir.iterdir()]) if data_dir.exists() else []
	raise FileNotFoundError(f"No se encontró ningún archivo con prefijo 'encuesta' en {data_dir.resolve()}. Archivos disponibles: {available}")

# preferir archivo .csv si existe, sino tomar el primero
csv_candidate = next((p for p in candidates if p.suffix.lower() == ".csv"), None)
file_to_read = csv_candidate or candidates[0]

# leemos el archivo csv y lo guardamos en un dataframe
df = pd.read_csv(file_to_read)
print(f"Leído: {file_to_read} (shape={df.shape})")

Leído: ..\data\encuesta100.csv (shape=(100, 10))


## 1.1 establecemos el rooting para que no suceda el mismo error de arriba de tener que no se encuentra el archivo y no se puede leer ###


## 2. Exploración y limpieza de datos ###
 Vemos el Dataframe y empezamos la limpieza como estandarizacion del mismo

In [21]:
df.shape  # mostramos la cantidad de filas y columnas del dataframe


(100, 10)

In [22]:
df.info()  # mostramos la informacion del dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   Fecha                                100 non-null    object
 1   Encuesta                             100 non-null    int64 
 2   Estrato                              100 non-null    object
 3   Sexo                                 100 non-null    object
 4   Edad                                 100 non-null    int64 
 5   Nivel Educativo                      100 non-null    object
 6   Cantidad de Integrantes en el Hogar  100 non-null    int64 
 7   Imagen del Candidato                 100 non-null    int64 
 8   Voto                                 100 non-null    object
 9   Voto Anterior                        100 non-null    object
dtypes: int64(4), object(6)
memory usage: 7.9+ KB


In [26]:
df.columns  # mostramos las columnas del dataframe

Index(['Fecha', 'Encuesta', 'Estrato', 'Sexo', 'Edad', 'Nivel Educativo',
       'Cantidad de Integrantes en el Hogar', 'Imagen del Candidato', 'Voto',
       'Voto Anterior'],
      dtype='object')

In [23]:
df.head()  # mostramos las primeras filas del dataframe

,Fecha,Encuesta,Estrato,Sexo,Edad,Nivel Educativo,Cantidad de Integrantes en el Hogar,Imagen del Candidato,Voto,Voto Anterior
0,2025-01-15,100,Alto,Femenino,33,Primario completo,2,87,Candidato A,No votó
1,2025-01-15,100,Bajo,Femenino,20,Primario incompleto,1,53,Candidato B,No votó
2,2025-01-15,100,Bajo,Masculino,63,Terciario/Universitario completo,6,74,En blanco,Candidato B
3,2025-03-01,400,Bajo,Femenino,69,Posgrado,1,88,Candidato B,En blanco
4,2025-02-15,300,Medio,Masculino,31,Posgrado,3,46,Candidato A,En blanco


In [24]:
df.tail()  # mostramos las ultimas filas del dataframe


,Fecha,Encuesta,Estrato,Sexo,Edad,Nivel Educativo,Cantidad de Integrantes en el Hogar,Imagen del Candidato,Voto,Voto Anterior
95,2025-02-01,200,Alto,Masculino,65,Secundario completo,3,86,No votó,No votó
96,2025-02-15,300,Alto,Masculino,36,Primario completo,3,51,Candidato C,Candidato A
97,2025-02-01,200,Medio,Masculino,80,Primario incompleto,5,58,Candidato B,En blanco
98,2025-01-15,100,Alto,Femenino,48,Secundario completo,4,61,Candidato B,Candidato A
99,2025-02-15,300,Medio,Masculino,70,Primario incompleto,4,71,Candidato A,No votó


In [27]:
df.sample(5)  # mostramos 5 filas aleatorias del dataframe

,Fecha,Encuesta,Estrato,Sexo,Edad,Nivel Educativo,Cantidad de Integrantes en el Hogar,Imagen del Candidato,Voto,Voto Anterior
34,2025-02-01,200,Alto,Femenino,31,Primario incompleto,5,87,No votó,Candidato A
11,2025-02-15,300,Alto,Masculino,70,Primario incompleto,3,65,Candidato C,Candidato A
74,2025-02-01,200,Medio,Masculino,35,Secundario completo,4,41,En blanco,Candidato C
48,2025-02-15,300,Alto,Masculino,60,Primario incompleto,3,89,No votó,En blanco
10,2025-02-01,200,Bajo,Masculino,28,Secundario completo,4,57,No votó,Candidato B


In [29]:
df.dtypes  # mostramos los tipos de datos de cada columna

Fecha                                  object
Encuesta                                int64
Estrato                                object
Sexo                                   object
Edad                                    int64
Nivel Educativo                        object
Cantidad de Integrantes en el Hogar     int64
Imagen del Candidato                    int64
Voto                                   object
Voto Anterior                          object
dtype: object

In [28]:
df.describe()  # mostramos estadisticas descriptivas del dataframe

,Encuesta,Edad,Cantidad de Integrantes en el Hogar,Imagen del Candidato
count,100.000000,100.000000,100.000000,100.000000
mean,250.000000,48.580000,3.420000,73.250000
std,108.711461,18.207103,1.694793,17.151361
min,100.000000,18.000000,1.000000,40.000000
25%,200.000000,33.000000,2.000000,58.750000
50%,200.000000,47.500000,3.000000,76.500000
75%,300.000000,65.250000,5.000000,88.250000
max,400.000000,80.000000,6.000000,100.000000


In [34]:
df_2 = df.copy()  # hacemos una copia del dataframe original para trabajar sobre ella
df_2.columns = [col.lower().strip().replace(" ", "_") for col in df_2.columns]
for col in df_2.select_dtypes(include='object').columns:
    df_2[col] = (
        df_2[col]
        .str.strip()                               # eliminar espacios al inicio/final
        .str.lower()                               # pasar a minúsculas
        .str.replace(r'[^a-z0-9\s]', '', regex=True)  # quitar caracteres especiales
        .fillna('0')                               # reemplazar NaN por '0'
    )
for col in df_2.select_dtypes(include=['int64', 'float64']).columns:
    df_2[col] = df_2[col].fillna(0)

## 3. Análisis estadistico ####
La idea es poder llevar adelante los siguientes temas:
- Distribución de la muestra
- Media, mediana y moda
- Test de hipotesis
- Regresiones 

 - Distribución de la Muestra